# **DATA PREPARATION**

## 1. Import Library & Load Dataset
- Mulai dari raw data
- Semua transformasi harus transparan dan reproducible

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load raw dataset
data_path = "../data/raw/solar-power.csv"
df = pd.read_csv(data_path)

df.head()

,Date-Hour(NMT),WindSpeed,Sunshine,AirPressure,Radiation,AirTemperature,RelativeAirHumidity,SystemProduction
0,01.01.2017-00:00,0.6,0,1003.8,-7.4,0.1,97,0.0
1,01.01.2017-01:00,1.7,0,1003.5,-7.4,-0.2,98,0.0
2,01.01.2017-02:00,0.6,0,1003.4,-6.7,-1.2,99,0.0
3,01.01.2017-03:00,2.4,0,1003.3,-7.2,-1.3,99,0.0
4,01.01.2017-04:00,4.0,0,1003.1,-6.3,3.6,67,0.0


## 2. Penanganan Kolom Waktu (Date-Hour)
**Tujuan**
- Menjadikan kolom waktu sebagai index
- Menjamin urutan data benar
- Mencegah data leakage

In [4]:
# Rename
df.rename(columns={"Date-Hour(NMT)": "datetime"}, inplace=True)

# Convert ke datetime
df["datetime"] = pd.to_datetime(df["datetime"], format="%d.%m.%Y-%H:%M")

df = df.sort_values("datetime")
# Jadikan index waktu
df.set_index("datetime", inplace=True)

df.head()

,WindSpeed,Sunshine,AirPressure,Radiation,AirTemperature,RelativeAirHumidity,SystemProduction
datetime,,,,,,,
2017-01-01 00:00:00,0.6,0,1003.8,-7.4,0.1,97,0.0
2017-01-01 01:00:00,1.7,0,1003.5,-7.4,-0.2,98,0.0
2017-01-01 02:00:00,0.6,0,1003.4,-6.7,-1.2,99,0.0
2017-01-01 03:00:00,2.4,0,1003.3,-7.2,-1.3,99,0.0
2017-01-01 04:00:00,4.0,0,1003.1,-6.3,3.6,67,0.0


## 3. Cek Struktur & Tipe Data
**Tujuan**
- Pastikan semua fitur numerik
- Tidak ada kolom aneh

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2017-01-01 00:00:00 to 2017-12-31 23:00:00
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   WindSpeed            8760 non-null   float64
 1   Sunshine             8760 non-null   int64  
 2   AirPressure          8760 non-null   float64
 3   Radiation            8760 non-null   float64
 4   AirTemperature       8760 non-null   float64
 5   RelativeAirHumidity  8760 non-null   int64  
 6   SystemProduction     8760 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 547.5 KB


## 4. Pemilahan Fitur (Feature Selection)
**Tujuan**
- Menentukan X (input) dan y (target)
- Menghindari kebocoran informasi

In [6]:
# tentukan target
target = 'SystemProduction'

# tentukan fitur input
features = [
    'WindSpeed',
    'Sunshine',
    'AirPressure',
    'Radiation',
    'AirTemperature',
    'RelativeAirHumidity'
]

In [7]:
# validasi
df[features + [target]].head()

,WindSpeed,Sunshine,AirPressure,Radiation,AirTemperature,RelativeAirHumidity,SystemProduction
datetime,,,,,,,
2017-01-01 00:00:00,0.6,0,1003.8,-7.4,0.1,97,0.0
2017-01-01 01:00:00,1.7,0,1003.5,-7.4,-0.2,98,0.0
2017-01-01 02:00:00,0.6,0,1003.4,-6.7,-1.2,99,0.0
2017-01-01 03:00:00,2.4,0,1003.3,-7.2,-1.3,99,0.0
2017-01-01 04:00:00,4.0,0,1003.1,-6.3,3.6,67,0.0


## 5. Cek Missing Value
**Tujuan**
- Menghindari error saat modeling
- Menjaga konsistensi data

In [8]:
df.isnull().sum()

WindSpeed              0
Sunshine               0
AirPressure            0
Radiation              0
AirTemperature         0
RelativeAirHumidity    0
SystemProduction       0
dtype: int64

## 6. Menangani Missing Value (Jika Ada)
**Prinsip**
- Jangan menghapus data sembarangan
- Gunakan pendekatan statistik sederhana

In [9]:
df[features] = df[features].fillna(df[features].median())
df[target] = df[target].fillna(0)

## 7. Analisis Nilai Nol (Konteks Energi Surya)
**Konsep penting**
- Produksi = 0 bukan error
- Biasanya terjadi malam hari

In [10]:
df["SystemProduction"].describe()

count    8760.000000
mean      684.746071
std      1487.454665
min         0.000000
25%         0.000000
50%         0.000000
75%       464.249950
max      7701.000000
Name: SystemProduction, dtype: float64

## 8. Deteksi Outlier

In [11]:
Q1 = df[features].quantile(0.25)
Q3 = df[features].quantile(0.75)
IQR = Q3 - Q1

outliers = ((df[features] < (Q1 - 1.5 * IQR)) |
            (df[features] > (Q3 + 1.5 * IQR))).sum()

outliers


WindSpeed               128
Sunshine               1894
AirPressure             232
Radiation              1149
AirTemperature            0
RelativeAirHumidity       2
dtype: int64

## 9. Pemisahan Fitur (X) dan Target (y)

In [12]:
X = df.drop(columns=["SystemProduction"])
y = df["SystemProduction"]

## 10. Time-Based Train-Test Split 
**Tujuan**
- Menghindari data leakage
- Simulasi prediksi masa depan

In [13]:
split_ratio = 0.7
split_index = int(len(df) * split_ratio)

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]

y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

## 11. Validasi Split

In [14]:
print("Train range:", X_train.index.min(), "to", X_train.index.max())
print("Test range :", X_test.index.min(), "to", X_test.index.max())

Train range: 2017-01-01 00:00:00 to 2017-09-13 11:00:00
Test range : 2017-09-13 12:00:00 to 2017-12-31 23:00:00


## 12. Simpan Data Bersih (Processed Data)
**Tujuan**
- Modeling & deployment tidak pakai raw data
- Menghindari pengulangan preprocessing

In [15]:
processed_path = "../data/processed/solar_power_clean.csv"
df.to_csv(processed_path)